# Getting Playlists using Spotify API w. Spotipy Library

I have the dataset from AICrowd's Spotify Sequential Skip Prediction Challenge, however the datasets are missing the original track ID which will help me identify which songs were skipped, and get a closer in depth view of the lyrics to the songs that were skipped. <p> In an attempt to try and rectify the missing information, I will pull all of the playlists that were created by Spotify (not user created) and then pull all the tracks with audio features from the Playlists and compare them to the Skip Challenge dataset. Should all go well, I will then scrape the lyrics to the songs from Genius and determine how much the artist's need to self announce at the beginning of the song plays into a user skipping the track. </p>

### Imports

In [9]:
df = pd.read_csv('./data/playlists.csv')
df['total_tracks'].sort_values(ascending = False)

1394    851
896     608
886     592
923     500
893     472
       ... 
637       6
1425      3
612       0
639       0
471       0
Name: total_tracks, Length: 1437, dtype: int64

In [10]:
df.loc[df['total_tracks'] == 851]

,playlist_name,playlist_id,total_tracks
1394,Sonisphere,2fC7gpl3V9MmCc8gBPHc66,851


In [1]:
import pandas as pd   

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials, SpotifyOAuth
import spotipy.util as util

### Setting up Credentials

In [2]:
token = util.prompt_for_user_token('222ugx5gr6cxvoyfbs5z2xpvy', # my user id on spotify
                                   client_id="28926b2395b54676a52b39cbd9477300", # my 'app' information
                                   client_secret="6acaf3dbedba475c8c2a5d431de42129",
                                   redirect_uri='http://localhost:8000/')
sp = spotipy.Spotify(auth=token) # getting the authorization token


In [3]:
# genres = [ "acoustic", "afrobeat", "alt-rock", "alternative", "ambient", "anime", "black-metal", "bluegrass", 
#                        "blues", "bossanova", "brazil", "breakbeat", "british", "cantopop", "chicago-house", "children", "chill",
#                        "classical", "club", "comedy", "country", "dance", "dancehall", "death-metal", "deep-house", 
#                        "detroit-techno", "disco", "disney", "drum-and-bass", "dub", "dubstep", "edm", "electro", "electronic", 
#                        "emo", "folk", "forro", "french", "funk", "garage", "german", "gospel", "goth", "grindcore", "groove", 
#                        "grunge", "guitar", "happy", "hard-rock", "hardcore", "hardstyle", "heavy-metal", "hip-hop", "holidays", 
#                        "honky-tonk", "house", "idm", "indian", "indie", "indie-pop", "industrial", "iranian", "j-dance", 
#                        "j-idol", "j-pop", "j-rock", "jazz", "k-pop", "kids", "latin", "latino", "malay", "mandopop", "metal", 
#                        "metal-misc", "metalcore", "minimal-techno", "movies", "mpb", "new-age", "new-release", "opera", 
#                        "pagode", "party", "philippines-opm", "piano", "pop", "pop-film", "post-dubstep", "power-pop", 
#                        "progressive-house", "psych-rock", "punk", "punk-rock", "r-n-b", "rainy-day", "reggae", "reggaeton", 
#                        "road-trip", "rock", "rock-n-roll", "rockabilly", "romance", "sad", "salsa", "samba", "sertanejo", 
#                        "show-tunes", "singer-songwriter", "ska", "sleep", "songwriter", "soul", "soundtracks", "spanish", 
#                        "study", "summer", "swedish", "synth-pop", "tango", "techno", "trance", "trip-hop", "turkish", 
#                        "work-out", "world-music" ]

In [4]:
# country_codes= ['AD', 'AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA', 'CL', 'CO', 'CR', 'CY', 'CZ', 'DK', 'DO', 'EC', 'SV', 
#                 'EE', 'FI', 'FR', 'DE', 'GR', 'GT', 'HN', 'HK', 'HU', 'IS', 'ID', 'IE', 'IT', 'JP', 'LV', 'LI', 'LT', 'LU', 
#                 'MY', 'MT', 'MX', 'MC', 'NL', 'NZ', 'NI', 'NO', 'PA', 'PY', 'PE', 'PH', 'PL', 'PT', 'SG', 'ES', 'SK', 'SE', 
#                 'CH', 'TW', 'TR', 'GB', 'US', 'UY']

trying different things before going into the function requests

In [5]:
# playlists = sp.user_playlists('spotify', limit=50, offset=0) # pulling playlists from spotify first 50
# playlists = sp.track('1tPGSiwtvmMVAqmniJ7oHp') # looking up a track 
# playlists = sp.recommendations(seed_artists='4NHQUGzhtTLFvgF5SZesLK') # getting recommendations based on seed track
# playlists['items'] # what's in here?

In [6]:
# Putting a loop together and checking outcome before trying a function

# s_playlists = sp.user_playlists('spotify', limit=50, offset= len(playlist_names))
# for i, playlist in enumerate(s_playlists['items']):
#     print(playlist['name'], '---', playlist['id'])
#     playlist_names.append(playlist['name'])
#     playlist_id.extend(playlist['id'])
    


testing out getting track info

In [7]:
# results = sp.user_playlist('spotify', '37i9dQZF1DXcBWIGoYBM5M', 'tracks')

In [8]:
# results['tracks']['items'][3]['track']['name']

### Playlist Fuction
Starts with 3 empty lists outside of the function,
- The function's purpose is to pull playlist names, IDs and the total number of tracks on the playlist, 
- the function takes a number of playlists argument. 
- Calls to the Spotipy / Spotify library using 'spotify' as the user_name, maximum pull is 50, offset is where to start, in the function we will start at 0 then, change it based on the length of playlist_names list, which will change as the new playlists are appended to it.
- For loop that iterates over the items & playlists in the playlist, pulls the name, id, track totals and appends them to their list.
- Additionally there is a 'progress report' every 50 pulls, which starts with a counter at 0, when the counter reaches 50 a print statement with the current length of the playlist_names is displayed and the counter is reset to 0.
- If a playlist has no data, a print statement will let me know.


In [9]:
# empty lists to append to later
playlist_names = [] 
playlist_id = [] 
total_tracks = []



def create_playlist(n_playlists): # function that takes the number of playlists to retrieve
    
    pulled = 1 # counter for progress report
    
    while len(playlist_names) < n_playlists: # while the length of playlist names is less than the number of playlists
        # make a call to Spotify using spotipy and get 50 playlists created by Spotify start at 0 offset and continue based on
        #current length of playlist_names until the playlist length requested is reached.
        s_playlists = sp.user_playlists('spotify', limit=50, offset=len(playlist_names)) 
        for i, playlist in enumerate(s_playlists['items']): # go through each item and playlist
            try:
                playlist_names.append(playlist['name']) # append name of playlist to list 
                playlist_id.append(playlist['id'])#append unique id of playlist to list 
                total_tracks.append(playlist['tracks']['total']) # append total number of tracks on playlist to list
                if pulled == 50: # give a progress report every 50 
                    print(f'There are now {len(playlist_names)} playlists')# Lets me know how many there are now.
                    pulled = 0 # reset counter
                pulled += 1 # add to counter
            except:
                print("NO DATA") #if the playlist has no data, it will print no data    
                

In [10]:
create_playlist(1437) # Initially I asked for 1500 but Spotify only has 1437 playlists that they created

There are now 50 playlists
There are now 100 playlists
There are now 150 playlists
There are now 200 playlists
There are now 250 playlists
There are now 300 playlists
There are now 350 playlists
There are now 400 playlists
There are now 450 playlists
There are now 500 playlists
There are now 550 playlists
There are now 600 playlists
There are now 650 playlists
There are now 700 playlists
There are now 750 playlists
There are now 800 playlists
There are now 850 playlists
There are now 900 playlists
There are now 950 playlists
There are now 1000 playlists
There are now 1050 playlists
There are now 1100 playlists
There are now 1150 playlists
There are now 1200 playlists
There are now 1250 playlists
There are now 1300 playlists
There are now 1350 playlists
There are now 1400 playlists


### Evaluating results
Now I will check to make sure everything came out the way I expect it

In [11]:
playlist_id[:15]

['37i9dQZF1DXcBWIGoYBM5M',
 '37i9dQZF1DX0XUsuxWHRQd',
 '37i9dQZF1DX1lVhptIYRda',
 '37i9dQZF1DX10zKzsJ2jva',
 '37i9dQZF1DX4JAvHpjipBk',
 '37i9dQZF1DX4dyzvuaRJ0n',
 '37i9dQZF1DX4SBhb3fqCJd',
 '37i9dQZF1DWXRqgorJj26U',
 '37i9dQZF1DX4sWSpwq3LiO',
 '37i9dQZF1DXcF6B6QPhFDv',
 '37i9dQZF1DWXJfnUiYjUKT',
 '37i9dQZF1DXcRXFNfZr7Tp',
 '37i9dQZF1DX4o1oenSJRJd',
 '37i9dQZF1DXbTxeAdrVG2l',
 '37i9dQZF1DX4UtSsGT1Sbe']

In [12]:
playlist_names[:15]

["Today's Top Hits",
 'RapCaviar',
 'Hot Country',
 '¡Viva Latino!',
 'New Music Friday',
 'mint',
 'Are & Be',
 'Rock Classics',
 'Peaceful Piano',
 'Rock This',
 'New Music Friday',
 'just hits',
 'All Out 00s',
 'All Out 90s',
 'All Out 80s']

In [13]:
total_tracks[:15]

[50, 50, 54, 50, 100, 65, 42, 145, 284, 50, 99, 77, 100, 100, 105]

### Dataframe and Save
Now I will save it to a dataframe with the lists as columns and then save to csv for use in the next notebook

In [14]:
playlist = pd.DataFrame(
    {'playlist_name': playlist_names,
     'playlist_id': playlist_id,
     'total_tracks': total_tracks
    })

In [15]:
playlist.head()

,playlist_name,playlist_id,total_tracks
0,Today's Top Hits,37i9dQZF1DXcBWIGoYBM5M,50
1,RapCaviar,37i9dQZF1DX0XUsuxWHRQd,50
2,Hot Country,37i9dQZF1DX1lVhptIYRda,54
3,¡Viva Latino!,37i9dQZF1DX10zKzsJ2jva,50
4,New Music Friday,37i9dQZF1DX4JAvHpjipBk,100


In [16]:
playlist.shape

(1437, 3)

In [17]:
#playlist.to_csv('./data/playlists.csv', index=False) # commented out so there is no override in the runs